In [2]:
# imports
import shutil
from pathlib import Path

from ipywidgets import IntProgress, Label, VBox
from IPython.display import display

In [3]:
# set network path and get list of directories to process
data_directory = Path('/Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR')
input_directory = data_directory.joinpath('1.toOCR')
output_directory = data_directory.joinpath('2.OCRed')

directories_to_convert_to_pdf_paths_list = sorted([x for x in input_directory.iterdir() if x.is_dir()])

print(f'{len(directories_to_convert_to_pdf_paths_list)} directories to process')

166 directories to process


In [5]:
# functions

def get_tif_paths_list(directory_path, scantailor=False):
    
    # if directory_path not a Path-like object then make it one
    if not isinstance(directory_path, Path):
        directory_path = Path(directory_path)
        
    # add 'out' path to directory if processed with ScanTailor
    if scantailor:
        directory_path = Path(directory_path).joinpath('out')
    
    # get paths to *.tif to process
    tif_paths_list = sorted(directory_path.glob('*.tif'))
    
    if len(tif_paths_list) == 0:
        tif_paths_list = sorted(directory_path.glob('*.tiff'))
    
    # only keep *.tif that DO NOT start with '.' to exclude macOS index files
    tif_paths_list = [x for x in tif_paths_list if not str(x.name).startswith('.')]
    
    return tif_paths_list


def create_temporary_pdf_directory(directory_path):
    
    # if directory_path not a Path-like object then make it one
    if not isinstance(directory_path, Path):
        directory_path = Path(directory_path)
        
    temporary_pdf_directory_path = Path.home().joinpath(f'Desktop/_temporary_pdfs_{directory_path.name}')
    
    try:  # creating the temporary directory
        temporary_pdf_directory_path.mkdir()
    except FileExistsError:  # delete the directory & all contents then create it
        shutil.rmtree(temporary_pdf_directory_path)
        temporary_pdf_directory_path.mkdir()
        
    return temporary_pdf_directory_path


def batch_ocr_tifs_into_pdfs(tif_paths_list, temporary_pdf_directory_path):
    
    number_of_tifs = len(tif_paths_list)
    number_of_pdfs = 0
    
    # progress bar
    progress_label = Label('OCR *.tif')
    progress_bar = IntProgress(min=0, max=number_of_tifs)
    progress_widget = VBox([progress_label, progress_bar])
    display(progress_widget)
    
    for tif_path in tif_paths_list:
        
        label = f'OCRing {tif_path.name}'
        progress_label.value = label

        temporary_pdf_output_path = temporary_pdf_directory_path.joinpath(tif_path.stem)

        # ocr *.tif with tesseract
        # NOTE: output SHOULD NOT have extension on end!
        !tesseract "{str(tif_path)}" "{str(temporary_pdf_output_path)}" pdf 2>/dev/null

        # Add the suffix .pdf that tesseract didn't want
        temporary_pdf_output_path = temporary_pdf_directory_path.joinpath(f'{tif_path.stem}.pdf') 

        if temporary_pdf_output_path.is_file():
            number_of_pdfs += 1
            progress_bar.value = number_of_pdfs
        
    if number_of_pdfs != number_of_tifs:
        print(f'ERROR - ERROR - ERROR')
        print(f'# of *.tif: {number_of_tifs}')
        print(f'# of PDFs: {number_of_pdfs}')
    else:
        print(f'{number_of_pdfs} PDFs created and saved in {temporary_pdf_directory_path}')
    
    
def concatenate_pdfs(temporary_pdf_directory_path):

    # strip '_temporary_pdfs_' from beginning of directory path to get combined PDF name
    combined_pdf_name = f"{str(temporary_pdf_directory_path).split('_temporary_pdfs_')[1]}.pdf"

    combined_pdf_output_path = temporary_pdf_directory_path.joinpath(combined_pdf_name)

    # concatenate with pdftk
    !pdftk "{str(temporary_pdf_directory_path)}"/*.pdf cat output "{str(combined_pdf_output_path)}"

    if combined_pdf_output_path.is_file() and combined_pdf_output_path.stat().st_size > 0:
        print(f'{combined_pdf_output_path} created with size {round((combined_pdf_output_path.stat().st_size/1024/1024/1024), 2)} GB')

    return combined_pdf_output_path

In [6]:
for dir_path in directories_to_convert_to_pdf_paths_list:
    
    # get list of *.tif to process
    tif_paths_list = get_tif_paths_list(dir_path)
    
    number_of_tifs = len(tif_paths_list)
    print(f'{number_of_tifs} *.tif to process in {dir_path}')
    
    # create temp directory
    temp_pdf_dir_path = create_temporary_pdf_directory(dir_path)
    
    # OCR tifs into PDFs with progress bar
    batch_ocr_tifs_into_pdfs(tif_paths_list, temp_pdf_dir_path)
    
    # concatenate PDFs into single file
    combined_pdf_path = concatenate_pdfs(temp_pdf_dir_path)
    
    # copy combined PDF to copy directory
    output_pdf_path = output_directory.joinpath(combined_pdf_path.name)
    shutil.copy(combined_pdf_path, output_pdf_path)
    
    if output_pdf_path.is_file():  # move processed dir_path into output_directory
        renamed_dir_path = output_directory.joinpath(dir_path.name)
        dir_path.rename(renamed_dir_path)

    print('')
    print('*****')    
    print(f'{dir_path.name} is now {renamed_dir_path}')
    print('*****')
    print('')

961 *.tif to process in /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/1.toOCR/R_ BOT_1958-02-24_EX1-33


961 PDFs created and saved in /Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1958-02-24_EX1-33
/Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1958-02-24_EX1-33/R_ BOT_1958-02-24_EX1-33.pdf created with size 0.25 GB

*****
R_ BOT_1958-02-24_EX1-33 is now /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/2.OCRed/R_ BOT_1958-02-24_EX1-33
*****

345 *.tif to process in /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/1.toOCR/R_ BOT_1958-05-09_EX1-126


345 PDFs created and saved in /Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1958-05-09_EX1-126
/Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1958-05-09_EX1-126/R_ BOT_1958-05-09_EX1-126.pdf created with size 0.03 GB

*****
R_ BOT_1958-05-09_EX1-126 is now /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/2.OCRed/R_ BOT_1958-05-09_EX1-126
*****

97 *.tif to process in /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/1.toOCR/R_ BOT_1958-06-19_EX1-14


97 PDFs created and saved in /Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1958-06-19_EX1-14
/Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1958-06-19_EX1-14/R_ BOT_1958-06-19_EX1-14.pdf created with size 0.02 GB

*****
R_ BOT_1958-06-19_EX1-14 is now /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/2.OCRed/R_ BOT_1958-06-19_EX1-14
*****

2 *.tif to process in /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/1.toOCR/R_ BOT_1958-09-11_EX1


2 PDFs created and saved in /Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1958-09-11_EX1
/Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1958-09-11_EX1/R_ BOT_1958-09-11_EX1.pdf created with size 0.0 GB

*****
R_ BOT_1958-09-11_EX1 is now /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/2.OCRed/R_ BOT_1958-09-11_EX1
*****

379 *.tif to process in /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/1.toOCR/R_ BOT_1958-10-17_EX1-227


...

372 PDFs created and saved in /Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1959-11-07_EX1-255
/Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1959-11-07_EX1-255/R_ BOT_1959-11-07_EX1-255.pdf created with size 0.06 GB

*****
R_ BOT_1959-11-07_EX1-255 is now /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/2.OCRed/R_ BOT_1959-11-07_EX1-255
*****

178 *.tif to process in /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/1.toOCR/R_ BOT_1960-01-25_EX1-11


^C
^C
^C
^C
^C
^C
^C
^C
^C
^C
^C
^C


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/dlisla/miniconda3/envs/dlisla/lib/python3.7/site-packages/IPython/utils/_process_posix.py", line 156, in system
    child = pexpect.spawn(self.sh, args=['-c', cmd])  # Vanilla Pexpect
  File "/Users/dlisla/miniconda3/envs/dlisla/lib/python3.7/site-packages/pexpect/pty_spawn.py", line 204, in __init__
    self._spawn(command, args, preexec_fn, dimensions)
  File "/Users/dlisla/miniconda3/envs/dlisla/lib/python3.7/site-packages/pexpect/pty_spawn.py", line 303, in _spawn
    cwd=self.cwd, **kwargs)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/dlisla/miniconda3/envs/dlisla/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-3f4e788d56b1>", line 13, in <module>
    batch_ocr_tifs_into_pdfs(tif_paths_list, temp_pdf_dir_path)
  File "<ipytho

UnboundLocalError: local variable 'child' referenced before assignment

In [ ]:
# open output_directory_path for visual Quality Assurance
!open {str(output_directory)}

In [4]:
# delete all '_temporary_pdfs_*' directories on the desktop
all_temp_dir_paths = sorted(Path('/Users/dlisla/Desktop/').glob('_temporary_pdfs_*'))
number_of_dir_paths = len(all_temp_dir_paths)
for temp_dir_path in all_temp_dir_paths:
    shutil.rmtree(temp_dir_path)

all_temp_dir_paths = sorted(Path('/Users/dlisla/Desktop/').glob('_temporary_pdfs_*'))
number_of_deleted_dir_paths = number_of_dir_paths - len(all_temp_dir_paths)

print(f'{number_of_deleted_dir_paths} _temporary_pdf_* directories deleted')

25 _temporary_pdf_* directories deleted
